# 🧠 Notebook: 02 ETL Gold Layer

This notebook implements the **Gold Layer (AI-Powered Analytics)** of the solution accelerator, transforming transcribed call data into enriched insights using **Databricks AI Functions** and **provisionless batch inference**.

It introduces structured AI outputs like sentiment, summaries, classifications, named entities, and even generates professional follow-up emails — ready for downstream workflows or customer engagement.

---

## 🧱 Purpose

To apply **advanced AI inference** on transcribed customer calls and output enriched, actionable insights in a format that can be:
- Embedded in dashboards
- Trigger customer communications
- Drive operational decisions

In [0]:
dbutils.library.restartPython()

In [0]:
%run "./resources/init" 

In [0]:
silver_df = spark.table(f"{CATALOG}.{SCHEMA}.{SILVER_TABLE}")
meta_path = f"{CATALOG}.{SCHEMA}.{META_TABLE}"

if spark._jsparkSession.catalog().tableExists(meta_path):
    processed_df = spark.table(meta_path).filter("processed = True")
    df = silver_df.join(processed_df, "file_name", "left_anti")
else:
    df = silver_df

if df.isEmpty():
    dbutils.notebook.exit("✅ No new files to process. Exiting Silver Layer.")

DataFrame[]

In [0]:
# Prompt template for ai_query()
prompt = """Using the following call transcript, generate a professional yet friendly email from the agent to the customer. The email should summarize the key points of the conversation, including the reason for the call, the resolution provided, and any necessary next steps for the customer. The tone should be courteous, clear, and supportive.

Email Structure:
- Subject Line: A clear and concise subject summarizing the purpose of the email (e.g., “Follow-up on Your Prescription Claim - VitalGuard Health Insurance”).
- Greeting: A friendly yet professional greeting addressing the customer by name.
- Call Summary: A recap of the discussion, including the inquiry and the response of the agent.
- Next Steps: A clear outline of any actions the customer needs to take (e.g., contacting their doctor, submitting forms, waiting for updates).
- Contact Information: An invitation for the customer to reach out if they have further questions.
- Closing: A polite and professional closing with the name of the agent and company details.

Call Transcript:
\n
"""

# Response schema for structured email output
response_format = '''{
    "type": "json_schema",
    "json_schema": {
        "name": "vitalguard_call_followup_email",
        "schema": {
            "type": "object",
            "properties": {
                "subject": {
                    "type": "string",
                    "description": "The subject line of the email summarizing the purpose of the follow-up."
                },
                "greeting": {
                    "type": "string",
                    "description": "A friendly yet professional greeting addressing the customer by name."
                },
                "call_summary": {
                    "type": "string",
                    "description": "A summary of the inquiry of the customer and the response given by the agent."
                },
                "next_steps": {
                    "type": "string",
                    "description": "Clear and concise next steps that the customer needs to take, if applicable."
                },
                "contact_information": {
                    "type": "string",
                    "description": "Details on how the customer can reach out for further assistance."
                },
                "closing": {
                    "type": "string",
                    "description": "A polite closing statement including the name of the agent and company details."
                }
            },
            "required": [
                "subject",
                "greeting",
                "call_summary",
                "next_steps",
                "contact_information",
                "closing"
            ]
        },
        "strict": true
    }
}'''

# Fetch call reason categories from lookup table
reasons_df = spark.table(f"{CATALOG}.{SCHEMA}.call_centre_reasons")
reasons_array = [f"'{row['reason_for_call']}'" for row in reasons_df.select("reason_for_call").distinct().collect()]
reasons_sql = ", ".join(reasons_array)

# Define NER targets
ner_targets = ["firstName_lastName", "dateOfBirth_yyyy-mm-dd", "policy_number"]
ner_sql = ", ".join([f"'{x}'" for x in ner_targets])

In [0]:
df.createOrReplaceTempView("transcriptions_temp")

query = f"""
SELECT *,
       ai_analyze_sentiment(transcription) AS sentiment,
       ai_summarize(transcription) AS summary,
       ai_classify(transcription, ARRAY({reasons_sql})) AS classification,
       ai_extract(transcription, ARRAY({ner_sql})) AS ner,
       ai_query('{ENDPOINT_NAME}', CONCAT('{prompt}', transcription), responseFormat => '{response_format}') AS email_response
FROM transcriptions_temp
"""

enriched_df = spark.sql(query)

# Add masking layer
enriched_df.createOrReplaceTempView("enriched_temp")

query_masked = """
SELECT *,
       ai_mask(summary, ARRAY('person', 'address')) AS summary_masked
FROM enriched_temp
"""

final_enriched_df = spark.sql(query_masked)

display(final_enriched_df)

file_path,file_name,transcription,audio_duration,call_id,agent_id,call_datetime,sentiment,summary,classification,ner,email_response,summary_masked
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10,"Hello, you've reached out Vital Guard, this is John speaking. How can I assist you today? Hi, this is Sophia Wilson. I was hoping to change my payment method. Of course, Sophia. For security, can I confirm your date of birth and policy number, please? Sure. My date of birth is 15th August 1995 and my policy number is VG924695. Thank you for confirming. What is your current payment method and how would you like to change it? I currently pay by direct debit, but I would like to switch to paying by credit card. Hmm, I can assist you with that. Please note that changing your payment method may affect your policy terms. Would you like me to explain the details? Yes, please do. Let me check that for you. Okay, I've updated your payment method to credit card. Your new payment schedule will be sent to you via email. Also, I'd like to take this opportunity to explain your coverage details. That sounds great, thank you. You're welcome. Just to summarize, I have changed your payment method to credit card and explained your coverage details. If you need further assistance, don't hesitate to reach out and have a wonderful day. Thanks, you too.",91.0,5e7e3k53,AGT002,2025-01-15T13:35:10Z,positive,Sophia Wilson changes her payment method to credit card and receives an explanation of her coverage details from John at Vital Guard.,Updating personal details,"List(Sophia Wilson, 1995-08-15, VG924695)","{""subject"": ""Follow-up on Payment Method Change - VitalGuard Health Insurance"", ""contact_information"": ""If you have any further questions or need assistance, please don't hesitate to contact me directly at john@vitalguard.com or call our customer service line at 1-800-VITALGUARD (1-800-848-2548)."", ""closing"": ""Best regards, John VitalGuard Health Insurance"", ""call_summary"": ""I am writing to follow up on our conversation earlier today regarding the change of your payment method from direct debit to credit card. As discussed, I have successfully updated your payment method, and you will receive a new payment schedule via email. Additionally, we reviewed your coverage details to ensure you have a clear understanding of your policy terms."", ""greeting"": ""Dear Sophia Wilson,"", ""next_steps"": ""Please review the new payment schedule carefully and contact us if you have any questions or concerns. If you need any further assistance or have questions about your policy, don't hesitate to reach out.""}",[MASKED] changes her payment method to credit card and receives an explanation of her coverage details from [MASKED] at Vital Guard.
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07,"Hello, this is Vital Guard, you are speaking to John, how can I help you today? Hi, this is Emma Johnson, I'm hoping to get some advice on my coverage options. Of course Emma, for security, can I confirm your date of birth and policy number please? Sure, my DOB is the 22nd of July 1990 and my policy number is VG 434271. Thank you for confirming. What kind of coverage details are you looking for? I want to know if my policy covers physiotherapy sessions. Okay, let me check that for you. Yes, your current plan includes up to 10 physiotherapy sessions per year, but they need to be prescribed by a GP. That's great to know. Also, if I need to make a complaint about a billing issue, how do I do that? I understand. I will provide you with a complaint reference number and you can submit your complaint through our online portal or over the phone. Here is your reference number, VG-CMP-4823. Thanks, that helps. Of course, you're very wel

In [0]:
from pyspark.sql.functions import col

flattened_df = final_enriched_df \
    .withColumn("customer_name", col("ner.firstName_lastName")) \
    .withColumn("birth_date", col("ner.dateOfBirth_yyyy-mm-dd")) \
    .withColumn("policy_number", col("ner.policy_number")) \
    .drop("modificationTime", "transcription", "ner")

display(flattened_df)

file_path,file_name,audio_duration,call_id,agent_id,call_datetime,sentiment,summary,classification,email_response,summary_masked,customer_name,birth_date,policy_number
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10,91.0,5e7e3k53,AGT002,2025-01-15T13:35:10Z,positive,Sophia Wilson changes her payment method to credit card and receives an explanation of her coverage details from John at Vital Guard.,Updating personal details,"{""subject"": ""Follow-up on Your Payment Method Change - VitalGuard Health Insurance"", ""contact_information"": ""If you have any further questions or need assistance, please don't hesitate to contact me directly at john@vitalguard.com or call our customer service line at 1-800-VITALGUARD (1-800-848-2548)."", ""closing"": ""Best regards,\nJohn\nVitalGuard Health Insurance"", ""call_summary"": ""I am writing to follow up on our conversation earlier today regarding the change of your payment method from direct debit to credit card. As discussed, I have successfully updated your payment method, and you will receive a new payment schedule via email. Additionally, we reviewed your coverage details to ensure you have a clear understanding of your policy terms."", ""greeting"": ""Dear Sophia Wilson,"", ""next_steps"": ""Please review the new payment schedule carefully and contact us if you have any questions or concerns. If there's anything else you need assistance with, don't hesitate to reach out.""}",[MASKED] changes her payment method to credit card and receives an explanation of her coverage details from [MASKED] at Vital Guard.,Sophia Wilson,1995-08-15,VG924695
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07,108.0,ct4m50n5,AGT005,2025-03-01T12:36:07Z,positive,"Emma inquires about physiotherapy coverage and billing complaints, receiving confirmation of 10 sessions per year and a complaint reference number.",Coverage details request,"{""subject"": ""Follow-up on Your Coverage Inquiry and Billing Issue - VitalGuard Health Insurance"", ""contact_information"": ""If you have any further questions or concerns, please do not hesitate to contact us at 1-800-VITALGUARD or through our website at vitalguard.com."", ""closing"": ""Thank you for choosing VitalGuard Health Insurance. Best regards, John, VitalGuard Health Insurance"", ""call_summary"": ""I am writing to follow up on our conversation earlier today regarding your coverage options and a billing issue. As discussed, your current policy includes up to 10 physiotherapy sessions per year, provided they are prescribed by a GP. Additionally, I provided you with a complaint reference number, VG-CMP-4823, for your billing issue, which you can use to submit your complaint through our online portal or over the phone."", ""greeting"": ""Dear Emma Johnson,"", ""next_steps"": ""To proceed with your physiotherapy sessions, please ensure you obtain a prescription from your GP. For your billing issue, you can now submit your complaint using the reference number provided. We will review your complaint and get back to you with an update.""}","[MASKED] inquires about physiotherapy coverage and billing complaints, receiving confirmation of 10 sessions per year and a complaint reference number.",Emma Johnson,1990-07-22,VG 434271
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,nv7032f9_AGT001_2025-02-27 12_40_45,114.0,nv7032f9,AGT001,2025-02-27T12:40:45Z,mixed,Michelle assists Liam with a duplicate policy document and explains his premium breakdown over the phone.,Coverage details request,"{""subject"": ""Follow-up on Your Duplicate Policy Document Request - VitalGuard Health Insurance"", ""contact_information"": ""If you have any further questions 

In [0]:
gold_table_path = f"{CATALOG}.{SCHEMA}.{GOLD_TABLE}"

if not spark._jsparkSession.catalog().tableExists(gold_table_path):
    flattened_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(gold_table_path)
else:
    flattened_df.write.mode("append").saveAsTable(gold_table_path)

print(f"✅ AI Enriched insights written to {GOLD_TABLE}")

✅ AI Enriched insights written to analysis_gold


In [0]:
from pyspark.sql.functions import lit

file_names_df = flattened_df.select("file_name").distinct().withColumn("processed", lit(True))

if spark._jsparkSession.catalog().tableExists(meta_path):
    current_meta_df = spark.table(meta_path)
    updated_meta_df = current_meta_df.unionByName(file_names_df).dropDuplicates(["file_name"])
else:
    updated_meta_df = file_names_df

# Overwrite metadata table with updated status
updated_meta_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(meta_path)

print(f"📌 Metadata table updated at: {meta_path}")

📌 Metadata table updated at: samantha_wise.ai_claims_processing_customer_demo.meta_data


## ✅ Output
**Table: analysis_gold**

Includes:
- AI sentiment classification
- Summaries (original and masked)
- Call reason classification
- Extracted customer info
- Structured JSON for follow-up emails

## ⏭ Next Step

Consume this enriched dataset in:
- **Dashboards** (e.g., sentiment trends, fraud alerts, agent metrics)
- **Case management systems**
- **Automated email or notification APIs**